In [41]:
import pandas as pd
import numpy as np

In [42]:
labels_paths = ["../data/积极1分数.xlsx", "../data/消极1分数.xlsx", "../data/积极2分数.xlsx", "../data/消极2分数.xlsx"]

In [43]:
def add_label(label_path, name):
    # 参数设定
    fs = 250                 # 采样率
    win_sec = 4              # 窗口长度（秒）
    step_sec = 1             # 步长（秒）
    label_interval = 4       # 原始标签每4秒一个
    label_len_samples = fs * label_interval

    labels = pd.read_excel(label_path, header=None).iloc[:, 1].to_numpy()

    # 拓展标签的时间范围（总时长）
    total_duration_sec = len(labels) * label_interval
    n_windows = (total_duration_sec - win_sec) // step_sec + 1

    # 创建新的窗口标签
    new_labels = []

    for i in range(n_windows):
        start_time = i * step_sec
        end_time = start_time + win_sec

        # 计算该窗口涉及哪些原始标签
        start_label_idx = start_time // label_interval
        end_label_idx = (end_time - 1e-6) // label_interval  # 减1e-6避免整数边界误差

        if start_label_idx == end_label_idx:
            new_labels.append(labels[int(start_label_idx)])
        else:
            # 跨越两个标签
            label1 = labels[int(start_label_idx)]
            label2 = labels[int(end_label_idx)]

            # 计算各自占比
            label1_end_time = (start_label_idx + 1) * label_interval
            label1_duration = label1_end_time - start_time
            label2_duration = end_time - label1_end_time

            weight1 = label1_duration / win_sec
            weight2 = label2_duration / win_sec

            weighted_avg = weight1 * label1 + weight2 * label2
            new_labels.append(weighted_avg)

    new_labels = np.array(new_labels)
    print(new_labels.shape)
    np.save(f"./y{name}.npy", new_labels)

In [44]:
for i in range(4):
    add_label(labels_paths[i], str(i+1))

(1193,)
(1193,)
(1025,)
(1313,)


In [45]:
y1 = np.load("y1.npy")
y2 = np.load("y2.npy")
y3 = np.load("y3.npy")
y4 = np.load("y4.npy")
# y = np.concatenate((y1, y2, y3, y4))
y = np.concatenate((y1[40:-20], y2[40:-20], y3[40:-20], y4[40:-20]))
np.save("y.npy", y)